In [48]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [49]:
import time
import numpy
from pandas import *
from tkinter import *

In [50]:
class Maze_Pars:
    def __init__(self, maze_world):
        file = open(maze_world, 'r')
        self.rowCol = file.readline()
        self.rowCol = self.rowCol.split(" ")
        self.agent = file.readline()
        self.agent = self.agent.split(" ")
        self.wumpus = file.readline()
        self.wumpus = self.wumpus.split(" ")
        self.gold = file.readline()
        self.gold = self.gold.split(" ")
        self.pits = []
        while True:
            pits = file.readline()
            if len(pits) == 0:
                break
            pits = pits.split(" ")
            self.pits.append(pits)

In [51]:
class Maze:
    def __init__(self):
        self.maze = [[]]
        
    def gen_maze(self, mazeWorld):
        # Instances to maze
        Maze_pars = Maze_Pars(mazeWorld)
        self.rows = int(Maze_pars.rowCol[0])
        self.cols = int(Maze_pars.rowCol[1])
        self.maze = [[[] for R in range(self.cols)] for C in range(self.rows)]
        self.agentRow = int(Maze_pars.agent[1])
        self.agentCol = int(Maze_pars.agent[2])
        self.maze[self.agentRow][self.agentCol].append('A')
        self.maze[int(Maze_pars.wumpus[1])][int(Maze_pars.wumpus[2])].append(Maze_pars.wumpus[0])
        self.maze[int(Maze_pars.gold[1])][int(Maze_pars.gold[2])].append(Maze_pars.gold[0])
        for pit in Maze_pars.pits:
            self.maze[int(pit[1])][int(pit[2])].append(pit[0])
        self.display_signs()

    # Displays indicators in maze
    # (W) Wumpus sign
    # (S) Stench sign
    # (P) Pits sign
    # (B) Breeze sign
    def display_signs(self):
        for R in range(self.rows):
            for C in range(self.cols):
                for RC in range(len(self.maze[R][C])):

                    # If maze row/col recognize WUMPUS
                    if self.maze[R][C][RC] == 'W':
                        if R-1 >= 0:
                            if 'S' not in self.maze[R-1][C]:
                                self.maze[R-1][C].append('S')
                        if C+1 < self.cols:
                            if 'S' not in self.maze[R][C+1]:
                                self.maze[R][C+1].append('S')
                        if R+1 < self.rows:
                            if 'S' not in self.maze[R+1][C]:
                                self.maze[R+1][C].append('S')
                        if C-1 >= 0:
                            if 'S' not in self.maze[R][C-1]:
                                self.maze[R][C-1].append('S')
                    
                    # If maze row/col recognize PIT
                    if self.maze[R][C][RC] == 'P':
                        if R-1 >= 0:
                            if 'B' not in self.maze[R-1][C]:
                                self.maze[R-1][C].append('B')
                        if C+1 < self.cols:
                            if 'B' not in self.maze[R][C+1]:
                                self.maze[R][C+1].append('B')
                        if R+1 < self.rows:
                            if 'B' not in self.maze[R+1][C]:
                                self.maze[R+1][C].append('B')
                        if C-1 >= 0:
                            if 'B' not in self.maze[R][C-1]:
                                self.maze[R][C-1].append('B')

In [52]:

class Agent:
    def __init__(self, maze, display_maze):
        # Instances of agent
        self.maze = maze
        self.mazeKnowledge = [[[] for R in range(self.maze.cols)] for C in range(self.maze.rows)]
        self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].append('A')
        self.stenches = 0

        # Cave exit = Agents starting row/col
        self.caveExit = [[self.maze.agentRow, self.maze.agentCol]]
        self.markTileVisited()

        # Cave entry row/col = Agents starting row/col
        self.maze.caveEntranceRow = self.maze.agentRow
        self.maze.caveEntranceCol = self.maze.agentCol
        
        # Creating boolean for methods
        self.mindedGold = False
        self.collectedGold = False
        self.exit = False

        self.display_maze = display_maze
        self.repaintMaze()

    def repaintMaze(self):
        # Changes the indicators in maze while moving
        for R in range(self.maze.rows):
            for C in range(self.maze.cols):
                textUpdate = []
                if 'A' in self.mazeKnowledge[R][C]:
                    textUpdate.append('A')
                if 'w' in self.mazeKnowledge[R][C]:
                    textUpdate.append('W')
                if 'p' in self.mazeKnowledge[R][C]:
                    textUpdate.append('P')
                if 'B' in self.mazeKnowledge[R][C]:
                    textUpdate.append('B')
                if 'S' in self.mazeKnowledge[R][C]:
                    textUpdate.append('S')
                if 'G' in self.mazeKnowledge[R][C]:
                    textUpdate.append('G')
                strUpdated=""
                self.display_maze[R][C].text_changed(strUpdated.join(textUpdate))
                if '.' in self.mazeKnowledge[R][C]:
                    self.display_maze[R][C].legend.config(bg="LightBlue")
                self.display_maze[R][C].legend.update()

    def move_back_tile(self):
        if self.maze.agentRow-1 == self.caveExit[-1][0]:
            self.move('up')
        if self.maze.agentRow+1 == self.caveExit[-1][0]:
            self.move('down')
        if self.maze.agentCol+1 ==  self.caveExit[-1][1]:
            self.move('right')
        if self.maze.agentCol-1 ==  self.caveExit[-1][1]:
            self.move('left')
        del self.caveExit[-1]

    def exit_cave(self):
        for tile in reversed(self.caveExit):
            if self.maze.agentRow-1 == tile[0]:
                self.move('up')
            if self.maze.agentRow+1 == tile[0]:
                self.move('down')
            if self.maze.agentCol+1 == tile[1]:
                self.move('right')
            if self.maze.agentCol-1 == tile[1]:
                self.move('left')
            if self.maze.caveEntranceRow == self.maze.agentRow:
                if self.maze.caveEntranceCol == self.maze.agentCol:
                    if self.mindedGold == True:
                        self.exit = True
                        break

    def explore_cave_for_gold(self):
        ifMoved = False
        while self.mindedGold == False:
            if self.mindedGold == True:
                break
            try:
                if  '.' not in self.mazeKnowledge[self.maze.agentRow-1][self.maze.agentCol] and self.agentSafeStep(self.maze.agentRow-1, self.maze.agentCol):
                    if ifMoved == False:
                        if self.move('up'):
                            ifMoved = True
            except IndexError:
                pass
            try:
                if '.' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol+1] and self.agentSafeStep(self.maze.agentRow, self.maze.agentCol+1):
                    if ifMoved == False:
                        if self.move('right'):
                            ifMoved = True
            except IndexError:
                pass
            try:
                if '.' not in self.mazeKnowledge[self.maze.agentRow+1][self.maze.agentCol] and self.agentSafeStep(self.maze.agentRow+1, self.maze.agentCol):
                    if ifMoved == False:
                        if self.move('down'):
                            ifMoved = True
            except IndexError:
                pass
            try:
                if '.' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol-1] and self.agentSafeStep(self.maze.agentRow, self.maze.agentCol-1):
                    if ifMoved == False:
                        if self.move('left'):
                            ifMoved = True
            except IndexError:
                pass
            if ifMoved == False:
                self.move_back_tile()
            ifMoved = False

    def move(self, direction):
        self.repaintMaze()
        if self.mindedGold == True and self.collectedGold == False:
            self.collectedGold == True
            if 'G' in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol]:
                self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].remove('G')
        stepSucces = False
        if direction == 'up':
            if self.agentSafeStep(self.maze.agentRow-1, self.maze.agentCol):
                stepSucces = self.stepUp()
        if direction == 'right':
            if self.agentSafeStep(self.maze.agentRow, self.maze.agentCol+1):
                stepSucces = self.stepRight()
        if direction == 'down':
            if self.agentSafeStep(self.maze.agentRow+1, self.maze.agentCol):
                stepSucces = self.stepDown()
        if direction == 'left':
            if self.agentSafeStep(self.maze.agentRow, self.maze.agentCol-1):
                stepSucces = self.stepLeft()
        if stepSucces:
            self.signsKnowledge()
            self.markTileVisited()
            self.estimateWumpus()
            self.estimatePits()
            self.cleanEstimation()
            self.reassureWumpusKnowledge()
            if 'G' in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol]:
                self.mindedGold = True
            if self.mindedGold == False:
                self.caveExit.append([self.maze.agentRow, self.maze.agentCol])
            time.sleep(1.5)
        return stepSucces

    def signsKnowledge(self):
        if 'B' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            if 'B' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol]:
                self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].append('B')
        if 'S' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            if 'S' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol]:
                self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].append('S')
        if 'G' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            if 'G' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol]:
                self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].append('G')
        if 'P' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            if 'P' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol]:
                self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].append('P')
        if 'W' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            if 'W' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol]:
                self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].append('W')

    def estimatePits(self):
        if 'B' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            if self.maze.agentRow-1 >= 0:
                if '.' not in self.maze.maze[self.maze.agentRow-1][self.maze.agentCol]:
                    if 'p' not in self.mazeKnowledge[self.maze.agentRow-1][self.maze.agentCol]:
                        self.mazeKnowledge[self.maze.agentRow-1][self.maze.agentCol].append('p')
        if 'B' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            if self.maze.agentCol+1 < self.maze.cols:
                if '.' not in self.maze.maze[self.maze.agentRow][self.maze.agentCol+1]:
                    if 'p' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol+1]:
                        self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol+1].append('p')
        if 'B' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            if self.maze.agentRow+1 < self.maze.rows:
                if '.' not in self.maze.maze[self.maze.agentRow+1][self.maze.agentCol]:
                    if 'p' not in self.mazeKnowledge[self.maze.agentRow+1][self.maze.agentCol]:
                        self.mazeKnowledge[self.maze.agentRow+1][self.maze.agentCol].append('p')
        if 'B' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            if self.maze.agentCol-1 >= 0:
                if '.' not in self.maze.maze[self.maze.agentRow][self.maze.agentCol-1]:
                    if 'p' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol-1]:
                        self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol-1].append('p')

    def estimateWumpus(self):
        if 'S' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            if self.maze.agentRow-1 >= 0:
                if '.' not in self.maze.maze[self.maze.agentRow-1][self.maze.agentCol]:
                    if 'w' not in self.mazeKnowledge[self.maze.agentRow-1][self.maze.agentCol]:
                        self.mazeKnowledge[self.maze.agentRow-1][self.maze.agentCol].append('w')
        if 'S' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            if self.maze.agentCol+1 < self.maze.cols:
                if '.' not in self.maze.maze[self.maze.agentRow][self.maze.agentCol+1]:
                    if 'w' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol+1]:
                        self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol+1].append('w')
        if 'S' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            if self.maze.agentRow+1 < self.maze.rows:
                if '.' not in self.maze.maze[self.maze.agentRow+1][self.maze.agentCol]:
                    if 'w' not in self.mazeKnowledge[self.maze.agentRow+1][self.maze.agentCol]:
                        self.mazeKnowledge[self.maze.agentRow+1][self.maze.agentCol].append('w')
        if 'S' in self.maze.maze[self.maze.agentRow][self.maze.agentCol]:
            if self.maze.agentCol-1 >= 0:
                if '.' not in self.maze.maze[self.maze.agentRow][self.maze.agentCol-1]:
                    if 'w' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol-1]:
                        self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol-1].append('w')

    def cleanEstimation(self):
        self.stenches = 0
        for R in range(self.maze.rows):
            for C in range(self.maze.cols):
                if 'S' in self.mazeKnowledge[R][C]:
                    self.stenches += 1
                if 'w' in self.mazeKnowledge[R][C]:
                    if C-1 >= 0:
                        if '.' in self.mazeKnowledge[R-1][C]:
                            if 'S' not in self.mazeKnowledge[R-1][C]:
                                self.mazeKnowledge[R][C].remove('w')
                    if C+1 < self.maze.cols:
                        if '.' in self.mazeKnowledge[R][C+1]:
                            if 'S' not in self.mazeKnowledge[R][C+1]:
                                self.mazeKnowledge[R][C].remove('w')
                    if R+1 < self.maze.rows:
                        if '.' in self.mazeKnowledge[R+1][C]:
                            if 'S' not in self.mazeKnowledge[R+1][C]:
                                self.mazeKnowledge[R][C].remove('w')
                    if C-1 >= 0:
                        if '.' in self.mazeKnowledge[R][C-1]:
                            if 'S' not in self.mazeKnowledge[R][C-1]:
                                self.mazeKnowledge[R][C].remove('w')

                if 'p' in self.mazeKnowledge[R][C]:
                    if R-1 >= 0:
                        if '.' in self.mazeKnowledge[R-1][C]:
                            if 'B' not in self.mazeKnowledge[R-1][C]:
                                self.mazeKnowledge[R][C].remove('p')
                    if C+1 < self.maze.cols:
                        if '.' in self.mazeKnowledge[R][C+1]:
                            if 'B' not in self.mazeKnowledge[R][C+1]:
                                self.mazeKnowledge[R][C].remove('p')
                    if R+1 < self.maze.rows:
                        if '.' in self.mazeKnowledge[R+1][C]:
                            if 'B' not in self.mazeKnowledge[R+1][C]:
                                self.mazeKnowledge[R][C].remove('p')
                    if C-1 >= 0:
                        if '.' in self.mazeKnowledge[R][C-1]:
                            if 'B' not in self.mazeKnowledge[R][C-1]:
                                self.mazeKnowledge[R][C].remove('p')

    def reassureWumpusKnowledge(self):
        for R in range(self.maze.rows):
            for C in range(self.maze.cols):
                if 'w' in self.mazeKnowledge[R][C]:
                    stenchesCloseBy = 0
                    if R-1 >= 0:
                        if 'S' in self.mazeKnowledge[R-1][C]:
                            stenchesCloseBy += 1
                    if C+1 < self.maze.cols:
                        if 'S' in self.mazeKnowledge[R][C+1]:
                            stenchesCloseBy += 1
                    if R+1 < self.maze.rows:
                        if 'S' in self.mazeKnowledge[R+1][C]:
                            stenchesCloseBy += 1
                    if C-1 >= 0:
                        if 'S' in self.mazeKnowledge[R][C-1]:
                            stenchesCloseBy += 1
                    if stenchesCloseBy < self.stenches:
                        self.mazeKnowledge[R][C].remove('w')

    def stepUp(self):
        if self.maze.agentRow-1 >= 0:
            self.clearAgent()
            self.maze.agentRow -= 1
            self.PutAgent()
            return True
        else:
            return False

    def stepRight(self):
        if self.maze.agentCol+1 < self.maze.cols:
            self.clearAgent()
            self.maze.agentCol += 1
            self.PutAgent()
            return True
        else:
            return False

    def stepDown(self):
        if self.maze.agentRow+1 < self.maze.rows:
            self.clearAgent()
            self.maze.agentRow += 1
            self.PutAgent()
            return True
        else:
            return False

    def stepLeft(self):
        if self.maze.agentCol-1 >= 0:
            self.clearAgent()
            self.maze.agentCol -= 1
            self.PutAgent()
            return True
        else:
            return False

    def clearAgent(self):
        self.maze.maze[self.maze.agentRow][self.maze.agentCol].remove('A')
        self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].remove('A')

    def PutAgent(self):
        self.maze.maze[self.maze.agentRow][self.maze.agentCol].append('A')
        self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].append('A')

    def markTileVisited(self):
        if '.' not in self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol]:
            self.maze.maze[self.maze.agentRow][self.maze.agentCol].append('.')
            self.mazeKnowledge[self.maze.agentRow][self.maze.agentCol].append('.')

    def agentSafeStep(self, row, col):
        if 'w' in self.mazeKnowledge[row][col]:
            return False
        if 'p' in self.mazeKnowledge[row][col]:
            return False
        if 'W' in self.mazeKnowledge[row][col]:
            return False
        if 'P' in self.mazeKnowledge[row][col]:
            return False
        return True



In [53]:
#----------------------------------- Grid legend ------------------------------------

class Display_Maze():
    def __init__(self, prime, R, C):
        self.text = StringVar()
        self.legend = Label(prime, textvariable = self.text, height = 6, width = 12, relief = GROOVE, bg = "Black", fg = "Blue", font = "Times 16 bold")
        self.legend.grid(row = R, column = C)
        self.row = R
        self.col = C
    def text_changed(self, textUpdate):
        self.text.set(str(textUpdate))

In [54]:
#----------------------------------- GUI  ------------------------------------

def solve_wumpus_maze(prime, maze_world):
    maze = Maze()
    maze.gen_maze(maze_world)
    display_maze = [[Display_Maze(prime, R, C) for C in range(maze.cols)] for R in range(maze.rows)]
    agent = Agent(maze, display_maze)
    # Agent Solving
    while agent.exit == False:
        agent.explore_cave_for_gold()
        if agent.mindedGold == True:
            agent.exit_cave()
        break
    agent.repaintMaze()
    agent.mazeKnowledge[agent.maze.agentRow][agent.maze.agentCol].remove('A')
    time.sleep(1.0)

prime = Tk()
prime.title("The Wumpus Game")
maze = Maze()
maze.gen_maze("Maze_1.txt")

display_maze = [[Display_Maze(prime, R, C) for C in range(maze.cols)] for R in range(maze.rows)]

startBTN = Button(prime, text="Start",  command= lambda: solve_wumpus_maze(prime, "Maze_1.txt"), font = "Times 16 bold", bg = "Black", fg = "Blue")
startBTN.grid(row = 0, column = len(display_maze[0]))

mainloop()